<h1 align=center><font size = 5>Capstone Project Week3: Segmenting and Clustering Neighborhoods in Toronto</font></h1>

<h2>Description</h2>
In this notebook, first time we will download data from url https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M  and extract data from this url to get table of data for Toronto and process following steps 1
Step 1: to noramalize and stard data downloaded Once the data is in a structured format, we can start the analysis to explore and cluster the neighborhoods in the city of Toronto.
Step 2: to cluster data in maps for data of Toronto
Step 3: examine data in clustered

#### Step 1: Download data and normalize data

In [1]:
#install some pre-libarary
!conda install -c conda-forge folium=0.5.0 --yes # For Foursquare API lab
!conda install -c conda-forge geopy --yes # For the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [34]:
#install some pre-libarary
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

import folium # map rendering library
from bs4 import BeautifulSoup
#import lxml
print('All of Libraries imported.')

All of Libraries imported.


Download scrape data from the following Wikipedia page and convert to pandas frame

In [86]:
# download data from url
url_data='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url_data)
#parse data use  BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')
#find table has class css'wikitable sortable'
table=soup.find('table', attrs={'class':'wikitable sortable'})

#convert html to pandas
table_html_string =str(table)
dfs = pd.read_html(table_html_string)
df=dfs[0]
df.head()


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Preprocess data to normalize and standdata

In [87]:

# ignore  boroughs is not assigned
df = df.drop(df[(df.Borough == "Not assigned")].index)

# set "Not assigned" Neighborhoods same name as Borough:
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
df.Neighborhood.fillna(df.Borough, inplace=True)

# remove duplicate rows:
df=df.drop_duplicates()

# parse string to extract title from column
df.update(
    df.Neighborhood.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

df.update(
    df.Borough.loc[
        lambda x: x.str.contains('title')
    ].str.extract('title=\"([^\"]*)',expand=False))

# delete Toronto comment from Neighbourhood:
df.update(
    df.Neighborhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace(", Toronto",""))
df.update(
    df.Neighborhood.loc[
        lambda x: x.str.contains('Toronto')
    ].str.replace("\(Toronto\)",""))


# to combine all neighborhoods that has same post code
df_data = pd.DataFrame({'Postcode':df.Postcode.unique()})
#for Neighborhood
df_data['Neighborhood']=pd.Series(list(set(df['Neighborhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df_data.iterrows())
df_data['Neighborhood']=df_data['Neighborhood'].apply(lambda x: ', '.join(x))
# for Borough
df_data['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df_data.iterrows())
df_data.dtypes

df_data.head()


,Postcode,Neighborhood,Borough
0,M3A,Parkwoods,North York
1,M4A,Victoria Village,North York
2,M5A,Harbourfront,Downtown Toronto
3,M6A,"Lawrence Manor, Lawrence Heights",North York
4,M7A,Queen's Park,Downtown Toronto


In [88]:
#download Geo-spatial data let us will get lat and lng from this data for above data
dfll= pd.read_csv("http://cocl.us/Geospatial_data")
print('Read data successully')
#rebame columns to be easty to code
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df_data.set_index("Postcode")
toronto_data=pd.merge(df_data, dfll)
toronto_data.head()

Read data successully


,Postcode,Neighborhood,Borough,Latitude,Longitude
0,M3A,Parkwoods,North York,43.753259,-79.329656
1,M4A,Victoria Village,North York,43.725882,-79.315572
2,M5A,Harbourfront,Downtown Toronto,43.654260,-79.360636
3,M6A,"Lawrence Manor, Lawrence Heights",North York,43.718518,-79.464763
4,M7A,Queen's Park,Downtown Toronto,43.662301,-79.389494


Get lat,lng of  Toronto use api  Nominatim:

In [89]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="my project")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Lat,Lng of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The Lat,Lng of Toronto, ON, Canada are 43.653963, -79.387207.


We create a map of Toronto with neighborhoods.

In [44]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### We are going to start using the Foursquare API to explore the neighborhoods and making segmentation them.

Set Foursquare Variables for making call Api 

In [90]:
CLIENT_ID = 'XGPWBTVFU2DVWDE33OUB2DB4ITA305P4FLBKN3OMCLKVZ0Z4' # your Foursquare ID
CLIENT_SECRET = 'VOOZVR5TAZXKQCKVACQPIL5GR43JV3QPDKV2TS0T2BE5MNFJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XGPWBTVFU2DVWDE33OUB2DB4ITA305P4FLBKN3OMCLKVZ0Z4
CLIENT_SECRET:VOOZVR5TAZXKQCKVACQPIL5GR43JV3QPDKV2TS0T2BE5MNFJ


Get and print the first neighborhood's name.

In [91]:
toronto_data.loc[0, 'Neighborhood']

'Parkwoods'

Get and print the neighborhood's latitude and longitude values.

In [92]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


We will get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [93]:
radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API
url_square = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
 # display URL
url_square


'https://api.foursquare.com/v2/venues/explore?&client_id=XGPWBTVFU2DVWDE33OUB2DB4ITA305P4FLBKN3OMCLKVZ0Z4&client_secret=VOOZVR5TAZXKQCKVACQPIL5GR43JV3QPDKV2TS0T2BE5MNFJ&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

Call the GET request and get the result of response by using above url_square

In [94]:
results_square = requests.get(url_square).json()
results_square

{'meta': {'code': 200, 'requestId': '5e1b488614a126001b269680'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.751976046055574,
          'lng': -79.33214044722958}],
        'distance': 245,
        'cc': 'CA'

We define function as **get_category_type** function like on the Foursquare lab.

In [95]:
# Define a function that will  extracts the category of the venue
# we copy from library open
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to normallize and standard data: clean the json and structure it into a *pandas* dataframe.

In [96]:
#filter by keyword items
venues = results_square['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter bycolumns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter by he category for each row using above functions
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean some columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# display information
nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,GTA Restoration,Fireworks Store,43.753396,-79.333477
2,Variety Store,Food & Drink Shop,43.751974,-79.333114
3,TTC stop - 44 Valley Woods,Bus Stop,43.755402,-79.333741


Print and see how many venues were returned by API Of Foursquare

In [97]:
#print result for data to see actual data
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


<a id='item2'></a>

#### Let start exploring Neighborhoods in Toronto

As theory of Square lab ,  we will create a function to repeat the same process to all the neighborhoods.

In [98]:
#borrow function from Square lab
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we will write the code to run the above function on each neighborhood ,this will create a new dataframe called *toronto_venues*.

In [99]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park
Queen's Park
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Ryerson, Garden District
Glencairn
West Deane Park, Cloverdale, Martin Grove, Islington, Princess Gardens
Port Union, Rouge Hill, Highland Creek
Don Mills South, Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Old Burnhamthorpe, Eringate, Markland Wood
West Hill, Guildwood, Morningside
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Richmond, Adelaide, King
Dovercourt Village, Dufferin
Scarborough Village
Oriole, Fairview, Henry Farm
York University, Northwood Park
East Toronto
Harbourfront East, Toronto Islands, Union Station
Trinity, Little Portugal
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
CFB Toronto, Downsview East
Riverdale, The Danfort

Let's examine the size of the resulting dataframe

In [100]:
print(toronto_venues.shape)
toronto_venues.head()

(2222, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


Let us see how many venues were printed for each neighborhood

In [101]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,54,54,54,54,54,54
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Bloordale Gardens, Old Burnhamthorpe, Eringate, Markland Wood",7,7,7,7,7,7
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16


Let us see how many unique categories can be selected  from all the returned venues

In [103]:
#print to view result
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


#### Analyze Data For Each Neighborhood

In [104]:
# make one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# to add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [105]:
#print shape for toronto_onehot
toronto_onehot.shape

(2222, 277)

Next, let's group rows by neighborhood and by caculaing the mean of the frequency of occurrence of each category

In [106]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#see result
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.000

Let's print and see the size of toronto_grouped

In [107]:
toronto_grouped.shape

(99, 277)

Let's print for debuging each neighborhood along with the top 5 most common venues

In [108]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1  Latin American Restaurant   0.2
2               Skating Rink   0.2
3                     Lounge   0.2
4             Breakfast Spot   0.2


----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1                   Pharmacy  0.05
2                Supermarket  0.05
3                 Restaurant  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                             venue  freq
0                             Bank  0.25
1                             Café  0.25
2               Chinese Restaurant  0.25
3              Japanese Restaurant  0.25
4  Molecular Gastronomy Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.08
1        Sandwich Place  0.08
2           Pizza Place  0.08
3    Italian Restaurant  0.08
4  Fast Food Restaurant  0.08


----Be

To convert all of data to a *pandas* dataframe

In [109]:
#Define an  function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

We will create  new dataframe and print the top 10 venues for each neighborhood.

In [110]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a  dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Clothing Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
1,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Diner,Fried Chicken Joint,Frozen Yogurt Shop,Shopping Mall,Deli / Bodega,Pizza Place,Pharmacy
2,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
3,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Fast Food Restaurant,Pizza Place,Italian Restaurant,Comfort Food Restaurant,Pub,Butcher,Sushi Restaurant,Indian Restaurant
4,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Bakery,Beer Bar,Steakhouse,Seafood Restaurant,Cheese Shop,Café,Breakfast Spot


#### Step 2:  Start  Clustering Neighborhoods For Toronto Data

Run *k*-means to cluster the neighborhood into 5 clusters.

In [111]:
# number of clusters
k_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [112]:
# to add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#copy from aboved toronto_data
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# examnine data
toronto_merged.head()


,Postcode,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,Parkwoods,North York,43.753259,-79.329656,1.0,Park,Bus Stop,Fireworks Store,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
1,M4A,Victoria Village,North York,43.725882,-79.315572,0.0,Grocery Store,Portuguese Restaurant,Coffee Shop,Hockey Arena,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Harbourfront,Downtown Toronto,43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Brewery,Shoe Store,Restaurant
3,M6A,"Lawrence Manor, Lawrence Heights",North York,43.718518,-79.464763,2.0,Clothing Store,Gift Shop,Boutique,Furniture / Home Store,Coffee Shop,Women's Store,Miscellaneous Shop,Vietnamese Restaurant,Accessories Store,Doner Restaurant
4,M7A,Queen's Park,Downtown Toronto,43.662301,-79.389494,2.0,Coffee Shop,Gym,Park,Yoga Studio,Diner,Beer Bar,Fast Food Restaurant,Italian Restaurant,Smoothie Shop,Sandwich Place


Make visualization for the resulting clusters

In [113]:
# use folium to create map
map_toronto_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add all of markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto_clusters)
#display map       
map_toronto_clusters

<a id='item5'></a>

#### Step 3: We will examine clusters

Now, you can examine each cluster from cluser 1 to k_clusters

#### Cluster 1

In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,0.0,Grocery Store,Portuguese Restaurant,Coffee Shop,Hockey Arena,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
32,Scarborough Village,0.0,Grocery Store,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
46,Downsview West,0.0,Grocery Store,Park,Bank,Hotel,Shopping Mall,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
60,Downsview Northwest,0.0,Gym / Fitness Center,Athletics & Sports,Liquor Store,Discount Store,Grocery Store,Airport Food Court,Diner,Farmers Market,Falafel Restaurant,Accessories Store
63,"The Junction North, Runnymede",0.0,Grocery Store,Caribbean Restaurant,Bus Line,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
83,"Summerhill East, Moore Park",0.0,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
89,"Jamestown, Mount Olive, Humbergate, South Stee...",0.0,Grocery Store,Fried Chicken Joint,Pharmacy,Sandwich Place,Pizza Place,Beer Store,Fast Food Restaurant,Liquor Store,Electronics Store,Eastern European Restaurant


#### Cluster 2

In [115]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,1.0,Park,Bus Stop,Fireworks Store,Food & Drink Shop,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
10,Glencairn,1.0,Park,Pub,Asian Restaurant,Japanese Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
16,Humewood-Cedarvale,1.0,Park,Field,Hockey Arena,Trail,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
21,Caledonia-Fairbanks,1.0,Women's Store,Park,Market,Fast Food Restaurant,Golf Course,Event Space,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Greek Restaurant
35,East Toronto,1.0,Park,Coffee Shop,Convenience Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,"CFB Toronto, Downsview East",1.0,Park,Airport,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
49,"Upwood Park, North Park, Downsview",1.0,Park,Construction & Landscaping,Basketball Court,Bakery,Ethiopian Restaurant,Empanada Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
61,Lawrence Park,1.0,Park,Swim School,Bus Line,Construction & Landscaping,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
64,Weston,1.0,Park,Convenience Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Department Store,Drugstore
66,York Mills West,1.0,Park,Bank,Convenience Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


#### Cluster 3

In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Harbourfront,2.0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Mexican Restaurant,Brewery,Shoe Store,Restaurant
3,"Lawrence Manor, Lawrence Heights",2.0,Clothing Store,Gift Shop,Boutique,Furniture / Home Store,Coffee Shop,Women's Store,Miscellaneous Shop,Vietnamese Restaurant,Accessories Store,Doner Restaurant
4,Queen's Park,2.0,Coffee Shop,Gym,Park,Yoga Studio,Diner,Beer Bar,Fast Food Restaurant,Italian Restaurant,Smoothie Shop,Sandwich Place
5,Queen's Park,2.0,Coffee Shop,Gym,Park,Yoga Studio,Diner,Beer Bar,Fast Food Restaurant,Italian Restaurant,Smoothie Shop,Sandwich Place
7,Don Mills North,2.0,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Café,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
8,"Parkview Hill, Woodbine Gardens",2.0,Fast Food Restaurant,Pizza Place,Gastropub,Pet Store,Pharmacy,Café,Breakfast Spot,Intersection,Bank,Athletics & Sports
9,"Ryerson, Garden District",2.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Japanese Restaurant,Theater,Pizza Place,Burger Joint,Sporting Goods Shop
12,"Port Union, Rouge Hill, Highland Creek",2.0,Bar,Golf Course,Moving Target,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
13,"Don Mills South, Flemingdon Park",2.0,Gym,Beer Store,Coffee Shop,Asian Restaurant,Clothing Store,Dim Sum Restaurant,Restaurant,Discount Store,Sporting Goods Shop,Italian Restaurant
14,Woodbine Heights,2.0,Park,Pharmacy,Beer Store,Curling Ice,Athletics & Sports,Asian Restaurant,Cosmetics Shop,Dance Studio,Skating Rink,Video Store


#### Cluster 4

In [117]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,"Emery, Humberlea",3.0,Baseball Field,Women's Store,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Dim Sum Restaurant


#### Cluster 5

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Malvern, Rouge",4.0,Fast Food Restaurant,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
